In [16]:
Today = '2024-04-30'
Mounth = 'Apr'

Print_ARAP = True
print_manual_map = True

import os
from datetime import date
import pandas as pd
import numpy as np

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [17]:
# query = """
# SELECT * 
# FROM "XXWH"."externalReceivableDebt"
# ORDER BY "reportDate" DESC
# FETCH FIRST 10 ROWS ONLY
# """
query = f"""
SELECT * 
FROM "XXWH"."externalReceivableDebt"
WHERE "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') 
"""

# invoiceDate
data = export_from_WHWEEK(query)
data_work_externalReceivableDebt = data[['companyCode', 'customerName', 'outGroupUS', 'currencyCode', 'amount', 'termDate']]
# Merge
data_work_externalReceivableDebt['Cty_code'] = merge_Mapping(data_work_externalReceivableDebt, 'customerName')
data_work_externalReceivableDebt['Cty_segment'] = merge_SalesUnits(data_work_externalReceivableDebt, 'Cty_code', merge_col='ocpSegment')
data_work_externalReceivableDebt['Comp_segment'] = merge_SalesUnits(data_work_externalReceivableDebt, 'companyCode', merge_col='ocpSegment')
data_work_externalReceivableDebt['Comp_holding'] = merge_SalesUnits(data_work_externalReceivableDebt, 'companyCode', merge_col='holding')
# Currency
data_work_externalReceivableDebt = add_in_currency_column(data_work_externalReceivableDebt, 'USD', 'currencyCode', False, 'amount', Today)
# Period
data_work_externalReceivableDebt = Period(data_work_externalReceivableDebt, Today, 'termDate')
# Pivot
data_pivot = data_work_externalReceivableDebt.pivot_table(index=['Comp_holding', 'companyCode', 'Comp_segment', 'outGroupUS'], values='amount_in_USD', aggfunc={'amount_in_USD':['sum','count']}).reset_index()
# to excel
if Print_ARAP == True:
    Output_file = f'{str(date.today())}_externalReceivableDebt_pivot_{Mounth}.xlsx'
    data_pivot.to_excel(Output_file, sheet_name='Pivot', index=False)
    new_list(data_work_externalReceivableDebt, Output_file, 'export_data')

query = f"""
SELECT * 
FROM "XXWH"."externalPayableDebt"
WHERE "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') 
"""
# invoiceDate
data = export_from_WHWEEK(query)
data_work_externalPayableDebt = data[['companyCode', 'vendorName', 'outGroupUS', 'currencyCode', 'amount', 'termDate']]
# Merge
data_work_externalPayableDebt['Cty_code'] = merge_Mapping(data_work_externalPayableDebt, 'vendorName')
data_work_externalPayableDebt['Cty_segment'] = merge_SalesUnits(data_work_externalPayableDebt, 'Cty_code', merge_col='ocpSegment')
data_work_externalPayableDebt['Comp_segment'] = merge_SalesUnits(data_work_externalPayableDebt, 'companyCode', merge_col='ocpSegment')
data_work_externalPayableDebt['Comp_holding'] = merge_SalesUnits(data_work_externalPayableDebt, 'companyCode', merge_col='holding')
# Currency
data_work_externalPayableDebt = add_in_currency_column(data_work_externalPayableDebt, 'USD', 'currencyCode', False, 'amount', Today)
# Period
data_work_externalPayableDebt = Period(data_work_externalPayableDebt, Today, 'termDate')
# Pivot
data_pivot = data_work_externalPayableDebt.pivot_table(index=['Comp_holding', 'companyCode', 'Comp_segment', 'outGroupUS'], values='amount_in_USD', aggfunc={'amount_in_USD':['sum','count']}).reset_index()
# to excel
if Print_ARAP == True:
    Output_file = f'{str(date.today())}_externalPayableDebt_pivot_{Mounth}.xlsx'
    data_pivot.to_excel(Output_file, sheet_name='pivot', index=False)
    new_list(data_work_externalPayableDebt, Output_file, 'export_data')


C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9916\1349408809.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_work_externalReceivableDebt['Cty_code'] = merge_Mapping(data_work_externalReceivableDebt, 'customerName')
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2

In [15]:
count = 0
for data_check in [data_work_externalReceivableDebt, data_work_externalPayableDebt]:
    count = count + 1
    if count == 1:
        cty_col = 'customerName'
    else:
        cty_col = 'vendorName'
    check_data = data_check.loc[(data_check['outGroupUS'] == 'Internal') & (data_check['Cty_segment'] == 'External'),\
                                                        [cty_col, 'Cty_code', 'Cty_segment']].drop_duplicates()
    check_data2 = data_check.loc[(data_check['outGroupUS'] == 'Internal') & (data_check['Comp_segment'] == 'External'),\
                                                        ['companyCode', 'Comp_segment']].drop_duplicates()
    check = pd.concat([check_data2, check_data])
    if len(check) != 0:
        if print_manual_map == True:
            check.to_excel(f'manual_map_Braz_{count}.xlsx', index=False)
    else:
       print(f'{count} is empty') 

1 is empty
